# MoRu Phase Prediction using GPyTorch Gaussian Process Classifier

In [ ]:
import torch
import gpytorch
import numpy as np
from matplotlib import pyplot as plt

### Read and Dump JSON

In [ ]:
import json

filename = 'data/MoRu_sparse.json'
stream = open(filename)
data = json.load(stream)
stream.close()

### Generate Dataset from JSON Dump

The dataset $\mathcal{D} \subset \mathbb{R}_{\ge 0}^{n_\text{samples} \times (n_\text{elements} + n_\text{phases} + 2)}$ is defined as:
$$\mathcal{D} = \{\mathbf{x}_i \, | \, i \in \{1, \dots, n_\text{samples}\}\}$$
where, $\mathbf{x}_i$ is a single sample such that:
$$\mathbf{x}_i = \left\{{T, P, \{c_j \, | \, j \in \mathcal{1, \dots, n_\text{elements}}\}, \{n_\phi \, | \, \phi \in \mathcal{1, \dots, n_\text{phases}}\}}\right\}$$
where, $c_j$ denotes number of moles of element $j$ and $n_\phi$ denotes the number of moles of phase $\phi$.

In [ ]:
n_data = len(data)
n_elements = len(data['2']['elements'])
n_phases = len(data['2']['solution phases']) + len(data['2']['pure condensed phases'])
dataset = np.zeros((n_data, n_elements + n_phases + 2))

phase_names = list(data['2']['solution phases'].keys())
phase_names += list(data['2']['pure condensed phases'].keys())
element_names = list(data['2']['elements'].keys())

In [ ]:
keys = list(data.keys())
for i in keys:
    dataset[keys.index(i), 0] = data[i]["temperature"]
    dataset[keys.index(i), 1] = data[i]["pressure"]

    for j in range(n_elements):
        if not (data[i]['elements'].get(element_names[j]) is None):
            dataset[keys.index(i), 2 + j] = data[i]["elements"][element_names[j]]["moles"]
        else:
            dataset[keys.index(i), 2 + j] = 0.0

    for j in range(n_phases):
        if not (data[i]['solution phases'].get(phase_names[j]) is None):
            dataset[keys.index(i), 2 + n_elements + j] = data[i]["solution phases"][phase_names[j]]["moles"]
        elif not (data[i]['pure condensed phases'].get(phase_names[j]) is None):
            dataset[keys.index(i), 2 + n_elements + j] = data[i]["pure condensed phases"][phase_names[j]]["moles"]
        else:
            dataset[keys.index(i), 2 + n_elements + j]

### Filter out zero phases

In [ ]:
moles_cutoff = 1e-8
non_zero_columns = np.any(np.greater(dataset[:,(2 + n_elements):], moles_cutoff), axis=0)

### Generate Training Data

- Training samples $n_\text{Train} = 0.6 * n_\text{Total}$
- Input $\mathbf{X} = {T, P, \mathbf{c}}$
- Output $\mathbf{y} = {\mathbf{n}_\phi}$

In [ ]:
n_training = int(n_data * 0.6)

choice = np.random.choice(range(dataset.shape[0]), size=(n_training,), replace=False)
train_indices = np.zeros(dataset.shape[0], dtype=bool)
train_indices[choice] = True
test_indices = ~train_indices

train = dataset[train_indices]
test = dataset[test_indices]

In [ ]:
# X_train = train[:,0:(2 + n_elements)]
X_train = train[:,0:(2 + n_elements)][:,(0,2)]                                  # Only T, Mo

# Change the outputs to 0.0 / 1.0 based on the number of moles of phase
# y_train = (train[:,(2 + n_elements):] > 0.0).astype(float)
y_train = (train[:,(2 + n_elements):][:,non_zero_columns] > 0.0).astype(float)  # Only non-zero phases

# Use the original moles 
# y_train = train[:,(2 + n_elements):]
# y_train = train[:,(2 + n_elements):][:,non_zero_columns]                        # Only non-zero phases


In [ ]:
# Convert to Torch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_train_unscaled = X_train.detach().clone()
y_train = torch.tensor(y_train, dtype=torch.float32)
y_train_unscaled = y_train.detach().clone()

In [ ]:
# Calculate mean and standard deviation of inputs
X_mean = X_train.mean(0,keepdim=True)
X_std = X_train.std(0,keepdim=True)

# Standard deviation of pressure is equal to 0.0. Change this to 1.0 to avoid division by zero when scaling
X_std[X_std == 0] = 1.0

# Scale inputs to mean 0 and standard deviation 1 for better convergence
X_train = (X_train - X_mean) / X_std

### Test Data

In [ ]:
# Scale test input values to mean 0 and standard deviation 1 based on values from before
# X_test_unscaled = torch.tensor(test[:,0:(2 + n_elements)], dtype=torch.float32)
X_test_unscaled = torch.tensor(test[:,0:(2 + n_elements)][:,(0,2)], dtype=torch.float32)      # Only T, Mo
X_test = (X_test_unscaled - X_mean) / X_std

# 0 / 1 Output case
# y_test = (test[:,(2 + n_elements):] > 0.0).astype(float)
y_test = (test[:,(2 + n_elements):][:,non_zero_columns] > 0.0).astype(float)                  # Only non-zero phases

# Original moles of phase
# y_test = test[:,(2 + n_elements):]
# y_test = test[:,(2 + n_elements):][:,non_zero_columns]                                      # Only non-zero phases
y_test = torch.tensor(y_test, dtype=torch.float32)

# Combined test and training data
X_all_unscaled = torch.cat([X_train_unscaled, X_test_unscaled])
X_all = torch.cat([X_train, X_test])
y_all = torch.cat([y_train, y_test])

### Input Space Mapping

In [ ]:
labels = ['Temperature', 'Moles_Mo']
n_columns = len(labels)

fig, ax = plt.subplots(nrows=n_columns, ncols=n_columns, figsize=(10, 10))

for i in range(n_columns):
  for j in range(n_columns):
    reduced, counts = np.unique(np.transpose(np.array([X_all_unscaled[:,i], X_all_unscaled[:,j]])), axis=0, return_counts=True)
    rx, ry = np.transpose(reduced)

    ax[i][j].scatter(rx, ry, c=counts, s=1)
    ax[i][j].set_xlabel(labels[i])
    ax[i][j].set_ylabel(labels[j])

fig.suptitle('Input Space Mapping\n')
fig.tight_layout()

### Softmax Normalization

Applies the Softmax function to output phase moles and rescales the moles to probabilities:
$$\text{Softmax}(x_i) = \frac{\exp{(x_i)}}{\sum_j \exp{(x_j)}}$$

In [ ]:
normalization = torch.nn.Softmax(dim=1)
y_train_normalized = normalization(y_train)
y_test_normalized = normalization(y_test)

### Multitask Model

In [ ]:
class MultiTaskGPModel(gpytorch.models.ExactGP):
  def __init__(self, train_x, train_y, likelihood):
    super(MultiTaskGPModel, self).__init__(train_x, train_y, likelihood)
    self.mean_module = gpytorch.means.MultitaskMean(gpytorch.means.ConstantMean(), num_tasks=5)
    self.covar_module = gpytorch.kernels.MultitaskKernel(gpytorch.kernels.RBFKernel(), num_tasks=5, rank=1)

  def forward(self, x):
    mean_x = self.mean_module(x)
    covar_x = self.covar_module(x)
    return gpytorch.distributions.MultitaskMultivariateNormal(mean_x, covar_x)
  
likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=5)
model = MultiTaskGPModel(X_train, y_train_normalized, likelihood)

### Training

In [ ]:
training_iterations = 1000

# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

for i in range(training_iterations):
    optimizer.zero_grad()
    output = model(X_train)
    loss = -mll(output, y_train_normalized)
    loss.backward()
    print(f'Iter{i+1}/{training_iterations} - Loss: {loss.item()}')
    optimizer.step()

### Predictions

In [ ]:
# Set to eval mode
model.eval()
likelihood.eval()

# Make predictions
with torch.no_grad(), gpytorch.settings.fast_pred_var():
  predictions = likelihood(model(X_test))
  means = predictions.mean
  lower, upper = predictions.confidence_region()

### Plots

In [ ]:
from itertools import compress

active_phases = list(compress(phase_names, non_zero_columns))

fig, axs = plt.subplots(nrows=1, ncols=len(active_phases), figsize=(25,5))

for i in range(len(active_phases)):
  ax  = axs[i]
  ax.set_xlabel('Mo')
  ax.set_ylabel('T')
  ax.set_title(active_phases[i])
  scatter = ax.scatter(X_test_unscaled[:,1].numpy(),
                       X_test_unscaled[:,0].numpy(),
                       c=(means[:,i]).numpy(), marker='.',
                       cmap='summer_r')
  fig.colorbar(scatter, orientation='horizontal', ax=ax)


In [ ]:
from matplotlib import colors

fig, axs = plt.subplots(nrows=1, ncols=len(active_phases), figsize=(25,5))
scatters = []

for i in range(len(active_phases)):
  axs[i].set_xlabel('Mo')
  axs[i].set_ylabel('T')
  axs[i].set_title(active_phases[i])
  scatters.append(axs[i].scatter(X_test_unscaled[:,1].numpy(), X_test_unscaled[:,0].numpy(),
                                 c=(means[:,i]).numpy(),
                                 marker='.',
                                 cmap='Blues'))

# Find the min and max of all colors for use in setting the color scale.
vmin = min(scatter.get_array().min() for scatter in scatters)
vmax = max(scatter.get_array().max() for scatter in scatters)
norm = colors.Normalize(vmin=vmin, vmax=vmax)
for sc in scatters:
    sc.set_norm(norm)

fig.colorbar(scatters[0], ax=axs, orientation='horizontal', fraction=.1)

# Make images respond to changes in the norm of other images (e.g. via the
# "edit axis, curves and images parameters" GUI on Qt), but be careful not to
# recurse infinitely!
def update(changed_scatter):
    for sc in scatters:
        if (changed_scatter.get_cmap() != sc.get_cmap()
                or changed_scatter.get_clim() != sc.get_clim()):
            sc.set_cmap(changed_scatter.get_cmap())
            sc.set_clim(changed_scatter.get_clim())

for sc in scatters:
    sc.callbacks.connect('changed', update)

plt.show()


### Error Analysis

In [ ]:
errors = np.zeros(y_test.shape)
fig, axs = plt.subplots(nrows=1, ncols=len(active_phases), figsize=(25,5))
cutoff = np.exp(0.0) / (4 + np.exp(1))

for i in range(len(active_phases)):
  # errors[:,i] = (np.logical_and(y_test[:, i].numpy() >= lower[:, i].numpy(),  y_test[:, i].numpy() <= upper[:, i].numpy())).astype(float)
  errors[:,i] = (np.logical_xor(y_test[:, i].numpy() >= cutoff, means[:, i].numpy() >= cutoff))
  ax  = axs[i]
  ax.set_xlabel('Mo')
  ax.set_ylabel('T')
  ax.set_title(active_phases[i])
  scatter = ax.scatter(X_test_unscaled[:,1].numpy(), X_test_unscaled[:,0].numpy(), c=errors[:,i], marker='.', cmap=colors.ListedColormap(['tab:green', 'tab:red']))
  fig.colorbar(scatter, orientation='horizontal', ax=ax)
  

In [ ]:
sorted_indices = (-means).numpy().argsort()
errors = np.zeros(y_test.shape)

for i in range(y_test.shape[0]):
  for j in range(y_test.shape[1]):
    if(y_test[i,j] > 0.0):                          # Present in full thermochemistry solve
      if(j in sorted_indices[i,(0,1)]):             # Present in surrogate (No error)
        errors[i,j] = 0.0
      else:                                         # Absent in surrogate (Hard error)
        errors[i,j] = 1.0
    else:                                           # Present in full thermochemistry solve
      if(j in sorted_indices[i,(0,1)]):
        errors[i,j] = 0.5                           # Present in surrogate (Soft error)
      else:
        errors[i,j] = 0.0                           # Absent in surrogate ()

errors_combined = np.max(errors, axis=1)
errors_combined_indices = np.argmax(errors, axis=1)

fig, axs = plt.subplots(nrows=1, ncols=len(active_phases), figsize=(25,5))
scatters = []

for i in range(len(active_phases)):
  axs[i].set_xlabel('Mo')
  axs[i].set_ylabel('T')
  axs[i].set_title(active_phases[i])
  scatters.append(axs[i].scatter(X_test_unscaled[:,1].numpy(), X_test_unscaled[:,0].numpy(),
                                 c=errors[:,i],
                                 marker='.',
                                 cmap=colors.ListedColormap(['tab:green', 'tab:orange' ,'tab:red'])))

# Find the min and max of all colors for use in setting the color scale.
vmin = min(scatter.get_array().min() for scatter in scatters)
vmax = max(scatter.get_array().max() for scatter in scatters)
norm = colors.Normalize(vmin=vmin, vmax=vmax)
for sc in scatters:
    sc.set_norm(norm)

fig.colorbar(scatters[0], ax=axs, orientation='horizontal', fraction=.1)

# Make images respond to changes in the norm of other images (e.g. via the
# "edit axis, curves and images parameters" GUI on Qt), but be careful not to
# recurse infinitely!
def update(changed_scatter):
    for sc in scatters:
        if (changed_scatter.get_cmap() != sc.get_cmap()
                or changed_scatter.get_clim() != sc.get_clim()):
            sc.set_cmap(changed_scatter.get_cmap())
            sc.set_clim(changed_scatter.get_clim())

for sc in scatters:
    sc.callbacks.connect('changed', update)

# for i in range(len(active_phases)):
#   ax  = axs[i]
#   ax.set_xlabel('Mo')
#   ax.set_ylabel('T')
#   ax.set_title(active_phases[i])
#   scatter = ax.scatter(X_test_unscaled[:,1].numpy(), X_test_unscaled[:,0].numpy(), c=errors[:,i], marker='.', cmap=colors.ListedColormap(['tab:green', 'tab:orange' ,'tab:red']))
#   fig.colorbar(scatter, orientation='horizontal', ax=ax)

In [ ]:
errors_combined.shape, errors.shape, y_test.shape

In [ ]:

fig = plt.figure(figsize = (6, 6))
ax = plt.axes()

ax.set_xlabel('Mo')
ax.set_ylabel('T')
ax.set_title('Maximum Error')
scatter = ax.scatter(X_test_unscaled[:,1].numpy(), X_test_unscaled[:,0].numpy(), c=errors_combined, marker='.', cmap='coolwarm')
fig.colorbar(scatter, orientation='horizontal', ax=ax)

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=len(active_phases), figsize=(25,10))
scatters_model = []
scatters_thermochimica = []

for i in range(len(active_phases)):
  axs[0,i].set_xlabel('Mo')
  axs[0,i].set_ylabel('T')
  axs[0,i].set_title(active_phases[i])
  scatters_model.append(axs[0,i].scatter(X_test_unscaled[:,1].numpy(), X_test_unscaled[:,0].numpy(),
                                 c=means[:,i],
                                 marker='.',
                                 cmap=colors.ListedColormap(['tab:green', 'tab:orange' ,'tab:red'])))
#   axs[0,i].tricontour(X_test_unscaled[:,1].numpy(), X_test_unscaled[:,0].numpy(), errors[:,i],  levels=1, linewidths=0.5)
  
  axs[1,i].set_xlabel('Mo')
  axs[1,i].set_ylabel('T')
  axs[1,i].set_title(active_phases[i])
  scatters_thermochimica.append(axs[1,i].scatter(X_test_unscaled[:,1].numpy(), X_test_unscaled[:,0].numpy(),
                                 c=y_test[:,i],
                                 marker='.',
                                 cmap=colors.ListedColormap(['tab:green', 'tab:orange' ,'tab:red'])))

# Find the min and max of all colors for use in setting the color scale.
vmin_ml = min(scatter.get_array().min() for scatter in scatters_model)
vmax_ml = max(scatter.get_array().max() for scatter in scatters_model)
norm_ml = colors.Normalize(vmin=vmin_ml, vmax=vmax_ml)
for sc in scatters_model:
    sc.set_norm(norm_ml)

vmin_tc = min(scatter.get_array().min() for scatter in scatters_thermochimica)
vmax_tc = max(scatter.get_array().max() for scatter in scatters_thermochimica)
norm_tc = colors.Normalize(vmin=vmin_tc, vmax=vmax_tc)
for sc in scatters_thermochimica:
    sc.set_norm(norm_tc)

# fig.colorbar(scatters_model[0], ax=axs, orientation='horizontal', fraction=.1)

# Make images respond to changes in the norm of other images (e.g. via the
# "edit axis, curves and images parameters" GUI on Qt), but be careful not to
# recurse infinitely!
def update(changed_scatter):
    for sc in scatters_model:
        if (changed_scatter.get_cmap() != sc.get_cmap()
                or changed_scatter.get_clim() != sc.get_clim()):
            sc.set_cmap(changed_scatter.get_cmap())
            sc.set_clim(changed_scatter.get_clim())

for sc in scatters_model:
    sc.callbacks.connect('changed', update)
for sc in scatters_thermochimica:
    sc.callbacks.connect('changed', update)

In [ ]:
from matplotlib import colors

fig, axs = plt.subplots(nrows=1, ncols=len(active_phases), figsize=(25,5))
scatters = []

for i in range(len(active_phases)):
  axs[i].set_xlabel('Mo')
  axs[i].set_ylabel('T')
  axs[i].set_title(active_phases[i])
  scatters.append(axs[i].scatter(X_test_unscaled[:,1].numpy(), X_test_unscaled[:,0].numpy(),
                                 c=(upper[:,i] - lower[:,i]).numpy(),
                                 marker='.',
                                 cmap='coolwarm'))
                                 

# Find the min and max of all colors for use in setting the color scale.
vmin = min(scatter.get_array().min() for scatter in scatters)
vmax = max(scatter.get_array().max() for scatter in scatters)
norm = colors.Normalize(vmin=vmin, vmax=vmax)
for sc in scatters:
    sc.set_norm(norm)

fig.colorbar(scatters[0], ax=axs, orientation='horizontal', fraction=.1)

# Make images respond to changes in the norm of other images (e.g. via the
# "edit axis, curves and images parameters" GUI on Qt), but be careful not to
# recurse infinitely!
def update(changed_scatter):
    for sc in scatters:
        if (changed_scatter.get_cmap() != sc.get_cmap()
                or changed_scatter.get_clim() != sc.get_clim()):
            sc.set_cmap(changed_scatter.get_cmap())
            sc.set_clim(changed_scatter.get_clim())

for sc in scatters:
    sc.callbacks.connect('changed', update)

In [ ]:
# Creating figure
sorted_indices = np.argsort(-means.numpy())

fig = plt.figure(figsize = (6, 6))
ax = plt.axes()

m1 = np.array([m[i] for m,i in zip(means, sorted_indices[:,0])])
m2 = np.array([m[i] for m,i in zip(means, sorted_indices[:,1])])
m3 = np.array([m[i] for m,i in zip(means, sorted_indices[:,2])])

ax.set_xlabel('Mo')
ax.set_ylabel('T')
ax.set_title(active_phases[i])
scatter = ax.scatter(X_test_unscaled[:,1].numpy(), X_test_unscaled[:,0].numpy(), c=(m3/m1), marker='.', cmap='coolwarm')
ax.scatter(error_points[1], error_points[0], facecolors='none', edgecolors='r')
fig.colorbar(scatter, orientation='horizontal', ax=ax)

plt.show()

In [ ]:
errors_combined = np.max(errors, axis=1)


error_points = X_test_unscaled[errors_combined == 1.0].numpy().transpose()
error_points

